In [1]:
from IPython.display import HTML
from dolfin import *
#import dijitso
#dijitso.set_log_level('debug')
import numpy
parameters["form_compiler"]["representation"] = "uflacs"
import matplotlib.pyplot as plt
%matplotlib inline
!cd ~/.cache/fenics && rm -r dijitso && cd -
#HTML(X3DOM().html(u))

/home/fenics/shared/trash


In [2]:
def change_size(mesh, x = 1, y = 1, z = 1):
    new_mesh = Mesh(mesh)
    new_mesh.coordinates()[:, 0] = x*mesh.coordinates()[:, 0]
    new_mesh.coordinates()[:, 1] = y*mesh.coordinates()[:, 1]
    #new_mesh.coordinates()[:, 2] = z*mesh.coordinates()[:, 2]
    return new_mesh

In [13]:
mesh = UnitQuadMesh(mpi_comm_world(), 75, 15)
mesh = UnitSquareMesh(75, 15)
mesh = change_size(mesh, x = 5)

In [14]:
laser_points = [numpy.linspace(-0.11, 7, 150), numpy.ones(150)]
laser_points = numpy.asarray(laser_points).T

In [15]:
# Define boundary func
def boundary(x, on_boundary):
    return on_boundary and near(x[1], 0.0, DOLFIN_EPS)

# Define variational problem
V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bc = DirichletBC(V, Constant(293.0), boundary) # Set dirichlet b.c. of value 0 to node set 'boundary'

u_0 = Expression('293.0', degree=1)
u_n = project(u_0, V)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [16]:
laser_center = Constant([0.5, 1])
laser_power = Constant(5e4)
laser_radius = Constant(0.1)
f = Expression("""sqrt(pow(x[0]-lc[0], 2) + pow(x[1]-lc[1], 2)) <= lr + tol
                  ? lp
                  : 0.0""", lp = laser_power, tol = DOLFIN_EPS, lc = laser_center, lr = laser_radius, degree = 1)

In [17]:
dt = 0.01
a = u*v*dx + dt*dot(grad(u), grad(v))*dx
L = (u_n + dt*f)*v*dx

In [18]:
# Create XDMF file for saving solution
file = XDMFFile("heat_solution_tri.xdmf")
file.parameters["rewrite_function_mesh"] = False

u1 = Function(V)
u1.rename("u1", "temperature")

# Time stepping
dt = 0.01
t = 0
num_steps = 150

A = assemble(a) # Assemble lhs
bc.apply(A)

for n in range(num_steps):
    
    # Update laser placement
    laser_center.assign(Constant(laser_points[n]))
    
    b = assemble(L) # Assemble the rhs vector
    
    bc.apply(b) # Apply boundary conditions
    
    # Solve linear system
    solve(A, u1.vector(), b )
    
    # Save current solution to file
    file.write(u1, t, XDMFFile.Encoding_ASCII)

    # Update previous solution
    u_n.assign(u1)
    
    # Update current time
    t += dt

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [19]:
HTML(X3DOM().html(u1))